In [ ]:
# Modules usuels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
# Pour lire les fichiers audios
import librosa, librosa.display
# Pour sauvegarder le modèle final
import joblib

In [ ]:
data = pd.read_csv('Transfos_data_prov.csv', sep = ";") # Fichier des features pour chaque audio

In [ ]:
data = data.sample(frac = 1) # Shuffle (sinon les fichiers modifiés artificiellement sont consécutifs dans le dataframe -> problèmes de biais)

## ACP

In [ ]:
data_pca=data.drop(columns=['ORIGIN-','SEXE-','ID']) # Préparation d'un dataframe en vue d'une ACP

In [ ]:
from sklearn import decomposition
from sklearn import preprocessing

n_comp = 2 # On veut 2 axes

X = data_pca.values
features = data_pca.columns

# Centrage et Réduction
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

# Calcul des composantes principales
pca = decomposition.PCA(n_components=n_comp)
pca.fit(X_scaled)


pcs = pca.components_
X_projected = pca.transform(X_scaled)
X=[]
Y=[]
for i in range(len(X_projected)):
    X.append(X_projected[i][0])
    Y.append(X_projected[i][1])
data_pca["X"]=X
data_pca["Y"]=Y

In [ ]:
# Affichage de l'ACP

import plotly.express as px
data_pca['SEXE-']=data['SEXE-']
fig=px.scatter(data_pca, x="X", y="Y",color='SEXE-')
fig


# RANDOM FOREST

In [ ]:
# Constitution des bases de données X_train, x_test, y_train, y_test
X_train=data[data['ORIGIN-']=='TRAIN'].drop(columns=['SEXE-','ORIGIN-','ID']).values
y_train=data[data['ORIGIN-']=='TRAIN']['SEXE-'].values
X_test=data[data['ORIGIN-']=='TEST'].drop(columns=['SEXE-','ORIGIN-','ID']).values
y_test=data[data['ORIGIN-']=='TEST']['SEXE-'].values

In [ ]:
# On charge un RandomForestClassifier de profondeur 10 (on cherchera les paramètres optimaux ensuite)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.96

On obtient des précisions élevées (**> 94 %**) sur l'ensemble de test

In [ ]:
# GridSearch, pour trouver le lot de paramètres
from sklearn.model_selection import GridSearchCV
# Grille des paramètres à tester
parameters = {'max_depth':[i for i in range(1,100,10)], 'n_estimators':[i for i in range(1, 100, 10)]}
rf = RandomForestClassifier()
clf = GridSearchCV(rf, parameters)
clf.fit(X_train, y_train)


sorted(clf.cv_results_.keys())

KeyboardInterrupt: 

In [ ]:
# Obtention du meilleur modèle
clf.best_estimator_

RandomForestRegressor(max_depth=80)

In [ ]:
# Utilisation des meilleurs paramètres (meilleur score)
clf = RandomForestClassifier(max_depth=5,n_estimators=110, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.94

In [ ]:
joblib.dump(clf, 'my_model.pkl', compress=9)

['my_model.pkl']